In [1]:
from selenium import webdriver
import bs4 as bs
import re
import pandas as pd
import os

In [2]:
# get the website
url_countries = 'https://www.transfermarkt.com/weltmeisterschaft-2022/teilnehmer/pokalwettbewerb/WM22/saison_id/2021?fbclid=IwAR1j_x1fk0AUukU-00X7Ip1atbJcjYeMf6ODfqLDUcwuU9bzWaW0WRkmdd8'
driver = webdriver.Chrome(executable_path='../../../Downloads/chromedriver-11')
driver.get(url_countries)
soup = bs.BeautifulSoup(driver.page_source,'lxml')

In [46]:
# extract participant countries
countries_urls = {}
for box in soup.find_all('div',  {'class' :'box'}):
    if 'Eliminated' in str(box) or 'Participating teams' in str(box):
        soup_box = bs.BeautifulSoup(str(box),'lxml')
        for country in soup_box.find_all('a', {'href': re.compile('verein/*')}):
            country_url = 'https://www.transfermarkt.com' + str(country).split('"')[1]
            country_name = country.text
            if len(country_name)>0:
                countries_urls[country_name] = country_url

df_countries_urls = pd.DataFrame(countries_urls.items(), columns = ['country_name', 'country_url'])
df_countries_urls

,country_name,country_url
0,England,https://www.transfermarkt.com/england/startsei...
1,Brazil,https://www.transfermarkt.com/brasilien/starts...
2,France,https://www.transfermarkt.com/frankreich/start...
3,Portugal,https://www.transfermarkt.com/portugal/startse...
4,Argentina,https://www.transfermarkt.com/argentinien/star...
5,Netherlands,https://www.transfermarkt.com/niederlande/star...
6,Croatia,https://www.transfermarkt.com/kroatien/startse...
7,Morocco,https://www.transfermarkt.com/marokko/startsei...
8,Germany,https://www.transfermarkt.com/deutschland/star...
9,Spain,https://www.transfermarkt.com/spanien/startsei...


In [122]:
# get the countries' players
folderout = 'countries_players'
if not os.path.exists(folderout):
    os.makedirs(folderout)
    
for country_name, country_url in countries_urls.items():
    
    driver = webdriver.Chrome(executable_path='../../../Downloads/chromedriver-11')
    driver.get(country_url)
    soup = bs.BeautifulSoup(driver.page_source,'lxml')

    for box in soup.find_all('div',  {'class' :'box'}):
        if 'Squad of'  in str(box):

            soup_box = bs.BeautifulSoup(str(box),'lxml')
            players  = soup_box.find_all('tr', {'class' : 'even'}) + soup_box.find_all('tr', {'class' : 'odd'})
            players_data = []

            for player in players:

                soup_player = bs.BeautifulSoup(str(player),'lxml')
                data = {}
                for s in soup_player.find_all('td'):
                    if 'tm-shirt-number' in str(s): data['tm-shirt-number'] = s.text
                    if 'marktwertverlauf' in str(s): data['market-value'] = s.text     
                    if 'athlete' in str(s): data['name'] = s.text        
                    if 'verein' in str(s): data['current-team'] = str(s).split('title=\"')[1].split('"')[0]
                    if 'hauptlink' in str(s):  data['url'] = 'https://www.transfermarkt.com' + str(s).split('href="')[-1].split('"')[0]

                players_data.append(data)


            df_players = pd.DataFrame(players_data)
            df_players['country'] = country_name


    df_players.to_csv(folderout + '/' + country_name + '_players.csv')      
    
    driver.quit()

In [ ]:
# get the players full history
dfs = pd.concat([pd.read_csv('countries_players/' + fn, index_col = 0) for fn in os.listdir('countries_players') if '.DS' not in fn])

options = webdriver.ChromeOptions()
options.add_argument("headless")    
    
folderout = 'players_history_raw'
if not os.path.exists(folderout):
    os.makedirs(folderout)
    

for pnd, (player_name, player_url) in enumerate(dfs.set_index('name').to_dict()['url'].items()):
    
    if not os.path.exists(folderout + '/' + player_name + '_raw.txt'):
    
        driver = webdriver.Chrome(executable_path='../../../Downloads/chromedriver-11', chrome_options=options)
        driver.get(player_url.replace('marktwertverlauf', 'transfers'))


        fout = open(folderout + '/' + player_name + '_raw.txt', 'w')
        fout.write(driver.page_source)
        fout.close()



        print(pnd, '   ', player_name, player_url)
    

<ipython-input-3-51eb8726300c>:16: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path='../../../Downloads/chromedriver-11', chrome_options=options)


314     Ismail Jakobs https://www.transfermarkt.com/ismail-jakobs/marktwertverlauf/spieler/375137
315     Moussa N'Diaye https://www.transfermarkt.com/moussa-ndiaye/marktwertverlauf/spieler/649022
316     Pape Gueye https://www.transfermarkt.com/pape-gueye/marktwertverlauf/spieler/485963
317     Mamadou Loum https://www.transfermarkt.com/mamadou-loum/marktwertverlauf/spieler/364237
318     Pape Matar Sarr https://www.transfermarkt.com/pape-matar-sarr/marktwertverlauf/spieler/568693
319     Pathé Ciss https://www.transfermarkt.com/pathe-ciss/marktwertverlauf/spieler/525578
320     Iliman Ndiaye https://www.transfermarkt.com/iliman-ndiaye/marktwertverlauf/spieler/623570
321     Nicolas Jackson https://www.transfermarkt.com/nicolas-jackson/marktwertverlauf/spieler/776890
322     Boulaye Dia https://www.transfermarkt.com/boulaye-dia/marktwertverlauf/spieler/626913
323     Famara Diédhiou https://www.transfermarkt.com/famara-diedhiou/marktwertverlauf/spieler/237125
324     Edouard Mendy htt

398     Ricardo Horta https://www.transfermarkt.com/ricardo-horta/marktwertverlauf/spieler/261927
399     André Silva https://www.transfermarkt.com/andre-silva/marktwertverlauf/spieler/198008
400     Cristiano Ronaldo https://www.transfermarkt.com/cristiano-ronaldo/marktwertverlauf/spieler/8198
401     Diogo Costa https://www.transfermarkt.com/diogo-costa/marktwertverlauf/spieler/357153
402     Rui Patrício https://www.transfermarkt.com/rui-patricio/marktwertverlauf/spieler/45026
403     António Silva https://www.transfermarkt.com/antonio-silva/marktwertverlauf/spieler/650568
404     Nuno Mendes https://www.transfermarkt.com/nuno-mendes/marktwertverlauf/spieler/616341
405     João Cancelo https://www.transfermarkt.com/joao-cancelo/marktwertverlauf/spieler/182712
406     Rúben Neves https://www.transfermarkt.com/ruben-neves/marktwertverlauf/spieler/225161
407     William Carvalho https://www.transfermarkt.com/william-carvalho/marktwertverlauf/spieler/100131
408     Vitinha https://www.t

480     Seung-gyu Kim https://www.transfermarkt.com/seung-gyu-kim/marktwertverlauf/spieler/92076
481     Kyung-won Kwon https://www.transfermarkt.com/kyung-won-kwon/marktwertverlauf/spieler/260189
482     Young-gwon Kim https://www.transfermarkt.com/young-gwon-kim/marktwertverlauf/spieler/126640
483     Chul Hong https://www.transfermarkt.com/chul-hong/marktwertverlauf/spieler/151791
484     Tae-hwan Kim https://www.transfermarkt.com/tae-hwan-kim/marktwertverlauf/spieler/156009
485     Jun-ho Son https://www.transfermarkt.com/jun-ho-son/marktwertverlauf/spieler/312446
486     In-beom Hwang https://www.transfermarkt.com/in-beom-hwang/marktwertverlauf/spieler/365394
487     Kang-in Lee https://www.transfermarkt.com/kang-in-lee/marktwertverlauf/spieler/557149
488     Heung-min Son https://www.transfermarkt.com/heung-min-son/marktwertverlauf/spieler/91845
489     Sang-ho Na https://www.transfermarkt.com/sang-ho-na/marktwertverlauf/spieler/508381
490     Woo-yeong Jeong https://www.transfer

565     Jonathan Osorio https://www.transfermarkt.com/jonathan-osorio/marktwertverlauf/spieler/260807
566     David Wotherspoon https://www.transfermarkt.com/david-wotherspoon/marktwertverlauf/spieler/74861
567     Junior Hoilett https://www.transfermarkt.com/junior-hoilett/marktwertverlauf/spieler/58993
568     Jonathan David https://www.transfermarkt.com/jonathan-david/marktwertverlauf/spieler/533738
569     Iké Ugbo https://www.transfermarkt.com/ike-ugbo/marktwertverlauf/spieler/339943
570     Andrew Redmayne https://www.transfermarkt.com/andrew-redmayne/marktwertverlauf/spieler/51975
571     Harry Souttar https://www.transfermarkt.com/harry-souttar/marktwertverlauf/spieler/298091
572     Bailey Wright https://www.transfermarkt.com/bailey-wright/marktwertverlauf/spieler/147419
573     Thomas Deng https://www.transfermarkt.com/thomas-deng/marktwertverlauf/spieler/366543
574     Joel King https://www.transfermarkt.com/joel-king/marktwertverlauf/spieler/616430
575     Nathaniel Atkinso

649     Mohammed Salisu https://www.transfermarkt.com/mohammed-salisu/marktwertverlauf/spieler/563963
650     Alexander Djiku https://www.transfermarkt.com/alexander-djiku/marktwertverlauf/spieler/250844
651     Alidu Seidu https://www.transfermarkt.com/alidu-seidu/marktwertverlauf/spieler/798404
652     Abdul-Rahman Baba https://www.transfermarkt.com/abdul-rahman-baba/marktwertverlauf/spieler/224884
653     Denis Odoi https://www.transfermarkt.com/denis-odoi/marktwertverlauf/spieler/59641
654     Salis Abdul Samed https://www.transfermarkt.com/salis-abdul-samed/marktwertverlauf/spieler/688707
655     Mohammed Kudus https://www.transfermarkt.com/mohammed-kudus/marktwertverlauf/spieler/543499
656     Kamaldeen Sulemana https://www.transfermarkt.com/kamaldeen-sulemana/marktwertverlauf/spieler/746695
657     Daniel Afriyie https://www.transfermarkt.com/daniel-afriyie/marktwertverlauf/spieler/754158
658     Osman Bukari https://www.transfermarkt.com/osman-bukari/marktwertverlauf/spieler/54

732     Riad Sharahili https://www.transfermarkt.com/riad-sharahili/marktwertverlauf/spieler/187951


In [155]:
with open(folderout + '/' + player_name + '_raw.txt') as mf:
    soup = bs.BeautifulSoup(mf.read(),'lxml')